In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

/content
--2024-02-08 18:45:42--  https://docs.google.com/uc?export=download&confirm=&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-
Resolving docs.google.com (docs.google.com)... 108.177.96.138, 108.177.96.101, 108.177.96.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.96.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-&export=download [following]
--2024-02-08 18:45:43--  https://drive.usercontent.google.com/download?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.96.132, 2a00:1450:4013:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.96.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35345757 (34M) [video/mp4]
Saving to: ‘vehicle-counting.mp4’

vehicle-counting.mp 100%[===================>]  33.71M   217

In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

In [ ]:
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass


@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False

In [ ]:
!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

In [ ]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

# Imports and download the video

In [ ]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from tqdm.notebook import tqdm
import os
from typing import List
import sys

HOME = os.getcwd()
print(HOME)
%cd {HOME}
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-" -O vehicle-counting.mp4 && rm -rf /tmp/cookies.txt

# Load the YOLOv5 model
model = YOLO('yolov5su.pt')

# Two main methods to get the color and position the tracker at the objects(Don't read them just use them)

In [ ]:
# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [ ]:
from sklearn.cluster import KMeans

def get_dominant_color(image, k=3):
    """
    Find the dominant color in an image using k-means clustering in the HSV color space.

    :param image: Image from which to get the dominant color (in BGR format).
    :param k: Number of clusters for k-means clustering.
    :return: Dominant color in HSV format.
    """
    # Convert to HSV
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    pixels = image_hsv.reshape((-1, 3))

    # Perform k-means clustering with an explicit n_init value
    clt = KMeans(n_clusters=k, n_init=10)  # Explicitly set n_init to suppress the warning
    clt.fit(pixels)

    # Assume the cluster with the highest weight has the dominant color
    hist = np.bincount(clt.labels_)
    dominant_color = clt.cluster_centers_[hist.argmax()]

    return dominant_color


def map_color_to_category(color_hsv):
    """
    Map an HSV color to a predefined color category, with improved handling for black and white.

    :param color_hsv: Color in HSV format (Hue, Saturation, Value).
    :return: Color category as a string.
    """
    # Handling for black and white
    if color_hsv[2] <= 50 and color_hsv[1] <= 50:  # Black: low value and saturation
        return "black"
    elif color_hsv[2] >= 200 and color_hsv[1] <= 25:  # White: high value and low saturation
        return "white"

    # Define color ranges in HSV format for other colors
    color_ranges = {
        "red": ((0, 50, 50), (10, 255, 255)),
        "blue": ((110, 50, 50), (130, 255, 255)),
        "green": ((50, 50, 50), (70, 255, 255)),
    }

    for color_name, (lower, upper) in color_ranges.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        if cv2.inRange(np.array([[color_hsv]], dtype=np.uint8), lower, upper).any():
            return color_name

    return "unknown"

def get_car_color(frame, bbox):
    """
    Determine the car's color from a video frame and bounding box.

    :param frame: The video frame (in BGR format).
    :param bbox: The bounding box of the detected car (in xyxy format).
    :return: The car's color as a string.
    """
    # Extract the region of interest from the frame
    x1, y1, x2, y2 = map(int, bbox)
    roi = frame[y1:y2, x1:x2]

    # Get the dominant color in the ROI
    dominant_color = get_dominant_color(roi, k=3)

    # Map the dominant color to a color category
    color_category = map_color_to_category(dominant_color)

    return color_category

# Read video and make predictions + output the video

In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck
CLASS_ID = [2, 3, 5, 7]

In [ ]:
# Store the targeted video
SOURCE_VIDEO_PATH = './vehicle-counting.mp4'
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

In [ ]:
# settings
LINE_START = Point(50, 1500)
LINE_END = Point(3840-50, 1500)
TARGET_VIDEO_PATH = f"{HOME}/vehicle-counting-result.mp4"

In [ ]:
# Initiate the tracker to track cars because yolo doesn't track car(tracker will give each car an ID)
byte_tracker = BYTETracker(BYTETrackerArgs())

# create frame generator
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

# create LineCounter instance to count the ins and outs
# First we need to set the line parameters based on the video's h,w
line_counter = LineCounter(start=LINE_START, end=LINE_END)

# create VideoInfo instance
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# create instance of BoxAnnotator and LineCounterAnnotator
line_annotator = LineCounterAnnotator(thickness=4, text_thickness=4, text_scale=2)
TARGET_VIDEO_PATH = f'{HOME}/counted_output_video.mp4'
color_counts = {}
tracked_car_ids = set()
# open target video file
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
      # create instance of BoxAnnotator
      box_annotator = BoxAnnotator(color=ColorPalette(), thickness=1, text_thickness=1, text_scale=1)

      # Make predictions
      results = model(frame)

      # model prediction on single frame and conversion to supervision Detections
      # This will prepare the boxes to be added to the frame
      detections = Detections(
          xyxy=results[0].boxes.xyxy.cpu().numpy(),
          confidence=results[0].boxes.conf.cpu().numpy(),
          class_id=results[0].boxes.cls.cpu().numpy().astype(int)
      )
      # tracking detections
      tracks = byte_tracker.update(
          output_results=detections2boxes(detections=detections),
          img_info=frame.shape,
          img_size=frame.shape
      )
      # Add the trackers numbers
      tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
      detections.tracker_id = np.array(tracker_id)

      # format custom labels with car color
      labels = [
          f"#{tracker_id}: {CLASS_NAMES_DICT[class_id]}, {get_car_color(frame, bbox)} {confidence:0.2f}"
          for bbox, confidence, class_id, tracker_id in zip(detections.xyxy, detections.confidence, detections.class_id, detections.tracker_id)
      ]


      for bbox, confidence, class_id, tracker_id in zip(detections.xyxy, detections.confidence, detections.class_id, detections.tracker_id):
          if tracker_id not in tracked_car_ids:
              color = get_car_color(frame, bbox)
              color_counts[color] = color_counts.get(color, 0) + 1
              tracked_car_ids.add(tracker_id)
              label = f"#{tracker_id}: {CLASS_NAMES_DICT[class_id]}, {color} {confidence:0.2f}"

      # updating line counter
      line_counter.update(detections=detections)

      # annotate and display frame
      frame = box_annotator.annotate(frame=frame, detections=detections, labels=labels)
      line_annotator.annotate(frame=frame, line_counter=line_counter)
      sink.write_frame(frame)

print("Color counts:")
for color, count in color_counts.items():
    print(f"{color}: {count}")